# Cleaning the datasets for Registro Público de Concesiones and zipcodes

This code includes all the cleaning done for the datasets that were scrapped and aggregates these into a big dataset.

- [Zipcodes](https://xn--cdigospostales-lob.es/listado-de-codigos-postales-de-espana/)
- [Main dataset](https://sedeaplicaciones.minetur.gob.es/RPC_Consulta)

In [231]:
import numpy as np
import pandas as pd

df = pd.read_csv('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/new_data/frequencies.csv')

df2 = df

# Import all the datasets for all regions
# region_1 = pd.read_csv('')
# region_2 = pd.read_csv('')
# region_3 = pd.read_csv('')
# region_4 = pd.read_csv('')
# region_5 = pd.read_csv('')
# region_6 = pd.read_csv('')
# region_7 = pd.read_csv('')
# region_8 = pd.read_csv('')
# region_9 = pd.read_csv('')
# region_10 = pd.read_csv('')
# region_11 = pd.read_csv('')
# region_12 = pd.read_csv('')
# region_13 = pd.read_csv('')
# region_14 = pd.read_csv('')
# region_15 = pd.read_csv('')
# region_16 = pd.read_csv('')
# region_17 = pd.read_csv('')
# region_18 = pd.read_csv('')
# region_19 = pd.read_csv('')

# Concat all the datasets
# df_concat = pd.concat([
#                         region_1,
#                         region_2,
#                         region_3,
#                         region_4,
#                         region_5,
#                         region_6,
#                         region_7,
#                         region_8,
#                         region_9,
#                         region_10,
#                         region_11,
#                         region_12,
#                         region_13,
#                         region_14,
#                         region_15,
#                         region_16,
#                         region_17,
#                         region_18,
#                         region_19,
#                         ])

# Clean all the region datasets
# df2 = df2.apply(lambda x: x.str.strip()).replace('', np.nan)
# df2 = df2.fillna(method='ffill')
# df2[['Frequencias', 'Tipo']] = df2['Frecuencia'].str.split(' ', 1, expand=True)
# del df2['Frecuencia']
# df2['Frequencias'] = df2['Frequencias'].apply(lambda x: x.replace('.', '')).apply(lambda x: x.replace(',', '.')).astype('float')

zipcodes = pd.read_excel('/Users/niko/Documents/Personal/GitHub/RadioLinkConcessionsSpain/new_data/listado-codigos-postales-con-LatyLon.xls')



In [377]:
""" 
    Doing a fuzzy join on the dataset of zipcodes and the data of reference
    data inside the 'Consulta del Registro Público de Concesiones'.
    This is needed, because the data from the source does not have zipcodes, longitute and latitude.
    The fuzzy join does a cross join, then calculates the fuzzy ratio and include the data based on a fuzzy_ratio.
    Then, the duplicated columns are dropped.
    The output is a new dataset that includes the zipcode, longitute and latitude per tower.
 """
 
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

# Import the packages neded
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define the fuzzy ratio used to include the data
fuzzy_ratio = 20

# Define the columns that we want to include in the final dataset
zipcodes_columns = ['codigopostalid','lat','lon']
df_columns = [ 'Referencia','Comunidad','Provincia','Municipio','Frequencias','Tipo']
all_new_columns = df_columns+zipcodes_columns

# Create new column to use to join both datasets
zipcodes['merge']='all'
df2['merge']='all'

# Join both datasets per row
all_datasets = pd.merge(df2,zipcodes,on='merge')
del all_datasets['merge']

# Create list of tuples based on the columns that we want to use for the join
datasets_tuple = all_datasets[['Municipio', 'poblacion']].apply(tuple, axis=1).tolist()

# Create the fuzz ratio on the list of tuples ceated
all_datasets['ratio'] = [fuzz.token_sort_ratio(*i) for i in datasets_tuple]

# Exclude those that have a low match ratio, the threshhold is set low because some matches have a low score
all_datasets = all_products[all_products.ratio>fuzzy_ratio]

# Drop all duplicates based on the defined columns and keep all the wanted ones
final_df = all_datasets[all_new_columns].drop_duplicates(subset=['Referencia','Municipio','Frequencias'])
final_df

,Referencia,Comunidad,Provincia,Municipio,Frequencias,Tipo,codigopostalid,lat,lon
0,ALAL-0700001,ANDALUCÍA,ALMERÍA,CUEVAS DEL ALMANZORA,15.019,GHz,240,-2.712437e+09,4.293981e+10
14665,ALAL-0700001,ANDALUCÍA,ALMERÍA,GALLARDOS (LOS),14.599,GHz,240,-2.712437e+09,4.293981e+10
29330,ALAL-0900386,ANDALUCÍA,ALMERÍA,ALMERIA,18875.000,MHz,240,-2.712437e+09,4.293981e+10
43997,ALAL-0900386,ANDALUCÍA,ALMERÍA,ROQUETAS DE MAR,17865.000,MHz,1450,-3.000073e+09,4.305428e+10
58660,ALAL-1000193,ANDALUCÍA,ALMERÍA,ALMERIA,14508.000,MHz,240,-2.712437e+09,4.293981e+10
73327,ALAL-1000193,ANDALUCÍA,ALMERÍA,TABERNAS,14928.000,MHz,1450,-3.000073e+09,4.305428e+10
87990,ALAL-1000250,ANDALUCÍA,ALMERÍA,ALMERIA,19.095,GHz,240,-2.712437e+09,4.293981e+10
102655,ALAL-1000250,ANDALUCÍA,ALMERÍA,ALMERIA,19.260,GHz,240,-2.712437e+09,4.293981e+10
117320,ALAL-1000250,ANDALUCÍA,ALMERÍA,ALMERIA,19.315,GHz,240,-2.712437e+09,4.293981e+10
131985,ALAL-1000250,ANDALUCÍA,ALMERÍA,ALMERIA,19.370,GHz,240,-2.712437e+09,4.293981e+10


## Cleaning the dataframe

        
    """if n%10 == 1:
        #Click second "..." if on greater than page 10
        if n < 20:
            driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[11]/a')[0].click()  
        else:
            driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[11]/a')[1].click()  
    else:
        driver.find_element_by_xpath("//td/a[text()='" + str(n)+ "']").click()    
    #Wait three seconds so the website doesn't crash
    driver.implicitly_wait(3)
"""
#Write to a csv
#df.to_csv("RegistroPublicoConcesiones_Frequencies.csv", index= False)

df["Municipio2"] = None
df["Frecuencia2"] = None

df["Municipio2"][0] = df["Municipio"][1]
df["Frecuencia2"][0] = df["Frecuencia"][1]

df

#Write to a csv
df.to_csv("RegistroPublicoConcesiones_Frequencies.csv", index= False, sep=";")